# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print (classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [23]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename, 'rb'))
    target = [i] * len(data)
    return data, target


In [24]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)
print('total number of data', len(all_data))

total number of data 222


In [25]:
print(all_data[0])

[-0.05058002471923828, -0.2269899845123291, -0.8298521041870117, 1.7180380821228027, -0.05058002471923828, -0.1487560272216797, -0.9311800003051758, 1.8945322036743164, -0.008992496877908707, -1.516520619392395]


In [5]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

In [6]:
print(training_data)

[ 44 196  19 209   0  87 189  46 100 184  79  64  24  56  39 156   7 120
  35 199 190 113 183  26 175 133  95   1 122 176 219 121  69  76 204  60
  38 155 150   5  97  73  84 216 129  74  81 101 201 158  52 174  90 215
 128 116 197 160   4 192  85 107 172 200  33 188  27  49 186  78  25 134
 206 181 163 218 108 127  28  13   3 212 198  40 208 169 159  93 214 147
 145 126 141  29  18 137 111  61  57  23  17   2  71  36 205 191 177 161
 119 109 103  94 157 154 203  70 211  43 135  32 104 110 102 114 106  30
 221 118  59 136  16 151  20  42 173 130 139  54  41 142 152  67  37  86
 210  34  68  11   8 180 217   9  10 105 168]


## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [7]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [8]:
# YOUR CODE HERE
train_data = []
train_target = []
test_data = []
test_target = []
for i in range(n_training_data):
    train_data.append(all_data[training_data[i]])
    train_target.append(all_target[training_data[i]])
for i in range(len(all_data)):
    if i not in training_data:
        test_data.append(all_data[i])
        test_target.append(all_target[i])
clf.fit(train_data, train_target)

SVC(C=100.0, gamma=0.001)

In [9]:
print(len(all_data) - len(test_data))
print(len(train_data))


155
155


### predicting

In [10]:
prediction = clf.predict(all_data[-1:]), all_target[-1:]

In [11]:
print(prediction)

(array([10]), [10])


In [12]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [13]:
expected = []
predicted = []
# YOUR CODE HERE
for i in range(len(all_data)):
    expected.append(all_target[i])

predicted = clf.predict(all_data)

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00        19
           2       0.97      1.00      0.98        30
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00        20
           7       1.00      0.91      0.95        11
           8       1.00      1.00      1.00        26
           9       1.00      1.00      1.00        11
          10       1.00      1.00      1.00        52

    accuracy                           1.00       222
   macro avg       1.00      0.99      0.99       222
weighted avg       1.00      1.00      1.00       222


Confusion matrix:
[[23  0  0  0  0  0  0  0  0  0  0]
 [ 0 19  0  0  0  0  0  0  0  0  0]
 [ 0  0 30  0  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [14]:
expected = []
predicted = []
# YOUR CODE HERE
for i in range(len(test_data)):
    expected.append(test_target[i])

predicted = clf.predict(test_data)

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         1
           2       0.92      1.00      0.96        12
           3       1.00      1.00      1.00         4
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         4
           6       1.00      1.00      1.00         3
           7       1.00      0.80      0.89         5
           8       1.00      1.00      1.00         8
           9       1.00      1.00      1.00         5
          10       1.00      1.00      1.00        14

    accuracy                           0.99        67
   macro avg       0.99      0.98      0.99        67
weighted avg       0.99      0.99      0.98        67


Confusion matrix:
[[ 6  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0]
 [ 0  0 12  0  0  0  0  0  0  0  0]
 [ 0  0  0  4  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [20]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [22]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict(all_data[:-1]), all_target[:-1]

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
         6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
         6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  2,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]),
 [0,
  0,
